In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from IPython.display import display
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-008f7ee07f23>", line 1, in <module>
    import matplotlib.pyplot as plt
  File "/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py", line 39, in <module>
    from matplotlib.figure import Figure, figaspect
  File "/opt/conda/lib/python3.6/site-packages/matplotlib/figure.py", line 40, in <module>
    from matplotlib.axes import Axes, SubplotBase, subplot_class_factory
  File "/opt/conda/lib/python3.6/site-packages/matplotlib/axes/__init__.py", line 4, in <module>
    from ._subplots import *
  File "/opt/conda/lib/python3.6/site-packages/matplotlib/axes/_subplots.py", line 10, in <module>
    from matplotlib.axes._axes import Axes
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_loa

KeyboardInterrupt: 

In [ ]:
cd ..

In [ ]:
%run src/load_data.py

In [ ]:
housing_df.head()
housing_df.shape

In [ ]:
def run_model(model_name, X_train, y_train, X_test, y_test):
    model = models[model_name]
    model.fit(X_train , y_train)
    return {
        'model_name' : model_name,
        'model_train_score' : model.score(X_train, y_train),
        'model_test_score' : model.score(X_test, y_test)
    }

In [ ]:
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
train_data =  data['ames']['train']['engineered']
train_labels = data['ames']['train']['labels']
test_data =  data['ames']['test']['engineered']
test_labels = data['ames']['test']['labels']

models = {
    'Ridge' : linear_model.Ridge(),
    #'Lasso' : linear_model.Lasso(),
    'K Nearest Neighbors' : KNeighborsRegressor(),
    'Decision Tree' : DecisionTreeRegressor(),
    'Support Vector Machines - RBF' : SVR(),
    'Support Vector Machines - Linear' : SVR(kernel ='linear')
}
results = []
for model_name in models.keys():
    results.append(run_model(model_name, train_data, train_labels, test_data, test_labels))

In [ ]:
#"Fitting data with very small alpha may cause precision problems. ConvergenceWarning)"
    'Lasso' : linear_model.Lasso(),

In [ ]:
results_df = pd.DataFrame(results)
results_df

In [ ]:
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

In [ ]:
models_params = {
    'Ridge' : {'alpha': range(1,50)},
    #'Lasso' : {'alpha': range(1,50)},
    'K Nearest Neighbors' : {'n_neighbors': range(1,20)},
    'Decision Tree' : {'max_depth': range(1,200)},
    'Support Vector Machines - RBF' : {'C': range(1,250,10)},
    'Support Vector Machines - Linear' : {'C': range(1,250,10)}
}
def run_model_grid_search(model_name, X_train, y_train, X_test, y_test):
    model = models[model_name]
    reg_params = models_params[model_name]
    model_gs = GridSearchCV(model, 
                      param_grid= reg_params,
                      cv=5,
                      return_train_score=True)
    model_gs.fit(X_train, y_train)
    return {
        'model_name' : model_name,
        'model_best_params' : model_gs.best_params_,
        'model_train_score' : model_gs.best_score_,
        'model_test_score' : model_gs.score(X_test, y_test)
    }

results = []
for model_name in tqdm(models.keys()):
    results.append(run_model_grid_search(model_name, train_data, train_labels, test_data, test_labels))
    results_df = pd.DataFrame(results)
results_df

In [ ]:
from time import time

In [ ]:
def time_function_call(function):
    def wrapper(*args, **kwargs):
        start = time()
        result = function(*args, **kwargs)
        execution_time = time() - start
        return result, execution_time
    return wrapper

@time_function_call
def model_fit(model, X, y):
    return model.fit(X, y)

@time_function_call
def model_predict(model, X):
    return model.predict(X)

def run_model(model, model_name, X_train, y_train, X_test, y_test):

    _, fit_time = model_fit(model, X_train, y_train)
    
    train_pred, train_pred_time = model_predict(model, X_train)
    
    test_pred, test_pred_time = model_predict(model, X_test)   
    
    return {
            'model' : model,
            'model_name' : model_name,
            'train_score' : model.score(X_train, y_train),
            'test_score' : model.score(X_test, y_test),
            'fit_time' : fit_time,
            'train_pred_time' : train_pred_time,
            'test_pred_time' : test_pred_time}

In [ ]:
test_results = []
alpha_values = range(1,20)
for ap in tqdm(alpha_values):
    model = KNeighborsRegressor(n_neighbors=ap)
    test_result = run_model(model,
                            'K Nearest Neighbors',
                            train_data, 
                            train_labels,
                           test_data,
                           test_labels)
    test_results.append(test_result)

In [ ]:
test_results_df = pd.DataFrame(test_results)
test_results_df['n'] = test_results_df.model.apply(lambda model: model.neighbors)

In [ ]:
plt.plot(test_results_df., test_results_df.train_score, label='train')
plt.plot(test_results_df., test_results_df.test_score, label='test')
plt.title('KNN Regression Score')
plt.ylabel('R2')
plt.xlabel('n')
plt.legend()

In [ ]:
test_results = []
alpha_values = range(1,400,20)
for ap in tqdm(alpha_values):
    model = SVR(kernel ='linear',C=ap)
    test_result = run_model(model,
                            'Support Vector Machines - Linear',
                            train_data, 
                            train_labels,
                           test_data,
                           test_labels)
    test_results.append(test_result)

In [ ]:
test_results_df = pd.DataFrame(test_results)
test_results_df['C'] = test_results_df.model.apply(lambda model: model.C)

In [ ]:
plt.plot(test_results_df.C, test_results_df.train_score, label='train')
plt.plot(test_results_df.C, test_results_df.test_score, label='test')
plt.title('SVM Regression Score')
plt.ylabel('R2')
plt.xlabel('C')
plt.legend()

In [ ]:
test_results = []
alpha_values = range(1,500)
for ap in tqdm(alpha_values):
    model = linear_model.Ridge(alpha=ap)
    test_result = run_model(model,
                            'Ridge',
                            train_data, 
                            train_labels,
                           test_data,
                           test_labels)
    test_results.append(test_result)

In [ ]:
test_results_df = pd.DataFrame(test_results)
test_results_df['alpha'] = test_results_df.model.apply(lambda model: model.alpha)

In [ ]:
plt.plot(test_results_df.alpha, test_results_df.train_score, label='train')
plt.plot(test_results_df.alpha, test_results_df.test_score, label='test')
plt.title('Ridge Regression Score')
plt.ylabel('R2')
plt.xlabel('alpha value')
plt.legend()